In [1]:
import pandas as pd
import numpy as np
import pickle, json
import random
import time
import os


import tensorflow as tf
from tensorflow import keras

from sklearn import linear_model
from sklearn.model_selection import train_test_split
import sklearn
import sklearn.datasets
from sklearn import preprocessing



from tqdm.notebook import tqdm

In [2]:
def extraction_dataset(data_2,number):
    test_model = data_2[data_2["Model"]==number]
    a = list(test_model["LayerNum"].value_counts().sort_index().keys())
    a_index = [i-1 for i in a]
    df_ = test_model.iloc[a_index,:]
    return df_

In [3]:
data = pd.read_excel(r"C:\Users\Samsung\jupyter\Project_PCB\data\Data_preprocessing5.xlsx")
data = data.fillna(0)
data.head()

,Model,LayerNum,PP,MatlMaker,RD,GD,MatlNum,RC,MatlG,Cu1,CP1,Cu2,CP2,Input_T,Thickness
0,1,1,1,DS7402M,1.36,2.56,1080.0,61.0,46.8,8.21,55,14.29,73,60,67.70
1,1,2,0,DS7402M,1.36,2.56,0.0,0.0,0.0,14.29,73,15.04,56,75,73.71
2,1,3,1,DS7402M,1.36,2.56,7628.0,47.0,203.4,15.04,56,14.58,46,200,187.25
3,1,4,0,DS7402M,1.36,2.56,0.0,0.0,0.0,14.58,46,14.29,75,75,77.47
4,1,5,1,DS7402M,1.36,2.56,3313.0,53.0,81.4,14.29,75,15.04,48,90,80.48


In [4]:
dic_glass = {}
keys = data["MatlG"].groupby(data["MatlNum"]).mean().keys()
values = data["MatlG"].groupby(data["MatlNum"]).mean().values
for i in range(len(keys)):
    dic_glass[keys[i]] = round(values[i],2)
data = data.drop(columns=["RD","GD","MatlNum"])


In [5]:
model_list = list(data["Model"].value_counts().keys())
testset_dic = {}
testset_list = []
for mo in model_list:
    globals () ["model_{}".format(mo)] = extraction_dataset(data,mo)
    testset_list.append(globals () ["model_{}".format(mo)])
    testset_dic["model_{}".format(mo)] = globals () ["model_{}".format(mo)]
drop_index = []
for i in testset_list:
    drop_index.extend(list(i.index))
data_drop = data.drop(drop_index)

In [6]:
data_drop = data_drop[data_drop["PP"]==1]
Maker = data_drop['MatlMaker'].unique()
Glass_all = data_drop['MatlG'].unique()
len_list = {}

for i in Maker:
    df = data_drop[data_drop['MatlMaker'] == i]
    Glass = df['MatlG'].unique()
    for j in Glass:
        df2 = df[df['MatlG'] == j]
        len_list["{} {}".format(i,j)] = len(df2)

# 'DS7402M 46.8' 로 test 하기

In [7]:
data_test = data_drop[data_drop["MatlMaker"]=="DS7402M"]
data_test = data_drop[data_drop["MatlG"]==46.8]

drop_list = ['MatlMaker','MatlG',"Model","PP"]
data_test = data_test.drop(columns=drop_list)
data_test.head()

,LayerNum,RC,Cu1,CP1,Cu2,CP2,Input_T,Thickness
11,1,61.0,8.37,55,14.29,73,60,66.19
21,11,61.0,14.29,73,8.53,56,60,65.44
22,1,61.0,8.69,55,14.29,73,60,67.70
32,11,61.0,14.29,73,8.05,56,60,66.02
33,1,61.0,8.37,55,13.54,73,60,67.70


In [8]:
data_x = data_test.values #returns a numpy array
min_max_scaler = preprocessing.Normalizer()
x_scaled = min_max_scaler.fit_transform(data_x)
data_2 = pd.DataFrame(x_scaled,columns=list(data_test.columns))
data_2.head()

,LayerNum,RC,Cu1,CP1,Cu2,CP2,Input_T,Thickness
0,0.007013,0.427806,0.058701,0.385726,0.100219,0.511964,0.420792,0.464204
1,0.076890,0.426392,0.099888,0.510273,0.059625,0.391442,0.419402,0.457428
2,0.006978,0.425638,0.060636,0.383772,0.099711,0.509370,0.418660,0.472388
3,0.076762,0.425683,0.099721,0.509424,0.056176,0.390791,0.418705,0.460715
4,0.006982,0.425911,0.058441,0.384018,0.094538,0.509697,0.418929,0.472691


In [9]:
y = data_2.loc[:,"Thickness"]
x = data_2.drop(columns="Thickness")
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [10]:
def rsq(y_pred,y_test):
    se = y_test - y_pred
    se2 = se ** 2
    s = sum(i for i in se2)

    st = y_test - np.mean(y_test)
    st2 = st ** 2
    t = sum(i for i in st2)

    r_sqr = 1 - (s/t)
    return r_sqr

In [11]:
tq =  tqdm(range(100))
n = 0
lr_result = []
for i in tq:
    X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=n)
    lr = linear_model.LinearRegression()
    lr.fit(X_train, y_train)
    lr_result.append(list(lr.coef_)+[lr.intercept_])
    n += 1
lr_df = pd.DataFrame(lr_result,columns=x.columns.to_list() + ["intercept"])
lr_statis = lr_df.describe()

  0%|          | 0/100 [00:00<?, ?it/s]

In [12]:
def rsq(y_pred,y_test):
    se = y_test - y_pred
    se2 = se ** 2
    s = sum(i for i in se2)

    st = y_test - np.mean(y_test)
    st2 = st ** 2
    t = sum(i for i in st2)

    r_sqr = 1 - (s/t)
    return r_sqr

tq =  tqdm(range(100))
n = 0
lr_result = []
for i in tq:
    X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=n)
    lr = linear_model.LinearRegression()
    lr.fit(X_train, y_train)
    lr_result.append(list(lr.coef_)+[lr.intercept_])
    n += 1
lr_df = pd.DataFrame(lr_result,columns=x.columns.to_list() + ["intercept"])
lr_statis = lr_df.describe()

  0%|          | 0/100 [00:00<?, ?it/s]

In [13]:
def initial_weight(lr_statis,layers_other):
    col_list = lr_statis.columns.to_list()
    w_all = []
    w_candi = []
    n_layers = layers_other[0]
    
    for col_ in col_list:
        mean_ = lr_statis.loc["mean",col_]
        std_ = lr_statis.loc["std",col_]
        candi_ = np.random.normal(mean_,std_,size=(1,n_layers))
        w_candi.extend(candi_)
    
    w_stack = np.stack(w_candi)
    w_all.append(w_stack)
    wei_2 = [np.random.normal(loc=0.0, scale=0.1, size=(y, x)) for x, y in zip(layers_other[:-1], layers_other[1:])]
    
    w_all.extend(wei_2)
    
    return w_all

In [14]:
WIE = initial_weight(lr_statis,[3,1])

# ANN-customize

In [15]:
def relu(x) : 
    x[x<=0] = 0
    return x

def reluDerivative(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

In [16]:

def sigmoid(t):
    return 1/(1+np.exp(-t))

# Derivative of sigmoid
def sigmoid_derivative(p):
    return p * (1 - p)

def relu(z):
    return np.maximum(0,z)

def d_relu(z):

    z[z>=0]=1
    z[z<0]= 0
    return z


def tanh(x):
    return np.tanh(x)

def d_tanh(x):
    return 1 - x * x


def none(z):
    return z

def d_none(z):
    return 1




class NeuralNetwork:
    def __init__(self, x,y,WIE,learn):
        self.input = x
        self.weights1= WIE[0]
        self.weights2 = WIE[1].T
        self.weights3 = WIE[2].T
        
        self.y = y
        self.alpha = learn
        self.output = np.zeros(y.shape)
        self.acti = sigmoid
        self.d_acti = sigmoid_derivative
        
        self.acti_out = sigmoid
        self.d_acti_out = sigmoid_derivative

    def feedforward(self):
        self.layer1 = self.acti(np.dot(self.input, self.weights1))
        self.layer2 = self.acti(np.dot(self.layer1, self.weights2))
        self.layer3 = self.acti_out(np.dot(self.layer2, self.weights3))
        
        return self.layer3
        
    def backprop(self):
        d_3 = 2*(self.y- self.output)*self.d_acti(self.output)
        d_2 = np.dot(d_3,self.weights3.T)*self.d_acti(self.layer2)
        d_1 = np.dot(d_2,self.weights2.T)*self.d_acti_out(self.layer1)
        
        d_weights3 = np.dot(d_3.T,self.layer2).T
        d_weights2 = np.dot(self.layer1.T,d_2)
        d_weights1 = np.dot(self.input.T,d_1)
        
        
        
        
        self.weights1 += self.alpha * d_weights1

        self.weights2 += self.alpha * d_weights2
        
        self.weights3 += self.alpha * d_weights3

    def train(self, X, y):
        self.output = self.feedforward()
        self.backprop()

In [17]:
X_train = X_train
X =np.stack([np.append(xt,[1]) for xt in X_train.values]).astype(np.float)

y = np.array(y_train).reshape(-1,1).astype(np.float)

In [18]:
WIE = initial_weight(lr_statis,[64,64,1])

In [19]:
NN = NeuralNetwork(X,y,WIE,0.01)
for i in range(150): # trains the NN 1,000 times
    if i % 10 ==0: 
        print ("for iteration # " + str(i) + "\n")
  
    NN.train(X, y)

for iteration # 0

for iteration # 10

for iteration # 20

for iteration # 30

for iteration # 40

for iteration # 50

for iteration # 60

for iteration # 70

for iteration # 80

for iteration # 90

for iteration # 100

for iteration # 110

for iteration # 120

for iteration # 130

for iteration # 140



In [20]:
NN.feedforward().mean()

0.46287127516248144

In [21]:
y.mean()

0.4638243600340797

In [22]:
np.array(y_test).reshape(-1,1).astype(np.float).mean()

0.4680597874871536

In [23]:
len(NN.feedforward())

462

In [48]:
# 데이터 정렬

def ANN_customize(data_drop,maker_name,glass_name):
    #glass_name_2= float(glass_name)
    data_test = data_drop[data_drop["MatlMaker"]==maker_name]
    data_test = data_drop[data_drop["MatlG"]==glass_name]

    drop_list = ['MatlMaker','MatlG',"Model","PP"]
    data_test = data_test.drop(columns=drop_list)

    data_x = data_test.values #returns a numpy array
    min_max_scaler = preprocessing.Normalizer()
    x_scaled = min_max_scaler.fit_transform(data_x)
    data_2 = pd.DataFrame(x_scaled,columns=list(data_test.columns))
    y = data_2.loc[:,"Thickness"]
    x = data_2.drop(columns="Thickness")

    ## LR
    tq =  tqdm(range(100))
    n = 0
    lr_result = []
    for i in tq:
        X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=n)
        lr = linear_model.LinearRegression()
        lr.fit(X_train, y_train)
        lr_result.append(list(lr.coef_)+[lr.intercept_])
        n += 1
    lr_df = pd.DataFrame(lr_result,columns=x.columns.to_list() + ["intercept"])
    lr_statis = lr_df.describe()

    # ANN
    X_train = X_train
    X =np.stack([np.append(xt,[1]) for xt in X_train.values]).astype(np.float)

    y = np.array(y_train).reshape(-1,1).astype(np.float)

    WIE = initial_weight(lr_statis,[64,64,1])
    NN = NeuralNetwork(X,y,WIE,0.01)
    for i in range(150): # trains the NN 1,000 times  
        NN.train(X, y)
    return NN.feedforward().mean()

In [49]:
ANN_customize(data_drop,'DS7402M',203.4)

  0%|          | 0/100 [00:00<?, ?it/s]

0.6845332565772377

In [50]:
ANN_customize(data_drop,'DS7402M',46.8)

  0%|          | 0/100 [00:00<?, ?it/s]

0.46579593845258843

In [25]:
WIE = initial_weight(lr_statis,[64,64,1])

In [26]:
### basic library
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from collections import Counter
### ETC
from tqdm.notebook import tqdm
import pathlib

### graph library
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import seaborn as sns
sns.set



### 분석관련 library
import statistics
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score,accuracy_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils.vis_utils import plot_model

print(tf.__version__)

2.4.0


In [27]:
## 데이터 불러오기
data_com = pd.read_excel(r"C:\Users\Samsung\jupyter\Project_PCB\data\Data_preprocessing5.xlsx")
## 결측값을 "0"으로 채우기
data_com = data_com.fillna(0)
data_com.head()

,Model,LayerNum,PP,MatlMaker,RD,GD,MatlNum,RC,MatlG,Cu1,CP1,Cu2,CP2,Input_T,Thickness
0,1,1,1,DS7402M,1.36,2.56,1080.0,61.0,46.8,8.21,55,14.29,73,60,67.70
1,1,2,0,DS7402M,1.36,2.56,0.0,0.0,0.0,14.29,73,15.04,56,75,73.71
2,1,3,1,DS7402M,1.36,2.56,7628.0,47.0,203.4,15.04,56,14.58,46,200,187.25
3,1,4,0,DS7402M,1.36,2.56,0.0,0.0,0.0,14.58,46,14.29,75,75,77.47
4,1,5,1,DS7402M,1.36,2.56,3313.0,53.0,81.4,14.29,75,15.04,48,90,80.48


In [28]:
## 사전만들기: MatlG를 통해서 MatlNum을 불러오기 위해
dic_glass = {}
keys = data_com["MatlG"].groupby(data_com["MatlNum"]).mean().keys()
values = data_com["MatlG"].groupby(data_com["MatlNum"]).mean().values
for i in range(len(keys)):
    dic_glass[keys[i]] = round(values[i],2)
## 사전만들기: 위랑 반대로 MatlNum을 통해서 MatlG를 불러올때
MatMaker_dic = {}
for i in range(len(data_com)):
    MatMaker_dic[data_com["MatlG"][i]] = data_com["MatlNum"][i]

In [29]:
## 사전 만들었으니 필요 없는 columns 지우기 
data_com = data_com.drop(columns=["RD","GD","MatlNum"])
data_com.head()

,Model,LayerNum,PP,MatlMaker,RC,MatlG,Cu1,CP1,Cu2,CP2,Input_T,Thickness
0,1,1,1,DS7402M,61.0,46.8,8.21,55,14.29,73,60,67.70
1,1,2,0,DS7402M,0.0,0.0,14.29,73,15.04,56,75,73.71
2,1,3,1,DS7402M,47.0,203.4,15.04,56,14.58,46,200,187.25
3,1,4,0,DS7402M,0.0,0.0,14.58,46,14.29,75,75,77.47
4,1,5,1,DS7402M,53.0,81.4,14.29,75,15.04,48,90,80.48


In [30]:
## 사용자함수(def_1): 각 Model 별 하나의 Blind testset(set: 1층~n층) 추출하기
def extraction_dataset(data_2,number):
    """
    data_2 = 전체 data
    number = 추출하려고 하는 model(:1~65)
    """
    ### 전체 data에서 model="number"인 것만 지정
    test_model = data_2[data_2["Model"]==number]
    
    ### 하나의 set를 가져오기 위해서, model에 있는 layer값들을 정렬(1층~...)
    a = list(test_model["LayerNum"].value_counts().sort_index().keys())
    a_index = [i-1 for i in a]
    ## 뽑아내기
    df_ = test_model.iloc[a_index,:]
    return df_

In [31]:
### model_list = model_1 ~ model_64 
model_list = list(data_com["Model"].value_counts().keys())


testset_dic = {}

### test_list:  뽑힌 정보의 index 정보를 저장하기 위해서 이용
#blinde testset으로 뽑힌 데이터는 train 할때 제거해야 하기 때문에,

testset_list = []
for mo in model_list:
    ### 각 model의 testset을 model_(n) 이름의 변수로 저장하기
    """
    사용자함수: extraction_dataset
    """
    globals () ["model_{}".format(mo)] = extraction_dataset(data_com,mo)
    testset_list.append(globals () ["model_{}".format(mo)])
    testset_dic["model_{}".format(mo)] = globals () ["model_{}".format(mo)]

In [32]:
data_com2 = data_com.copy()
data_com2["MatlMaker_cate"] = data_com2["MatlMaker"].astype("category")
cat_col = data_com2.select_dtypes(["category"]).columns
data_com2[cat_col] = data_com2[cat_col].apply(lambda x: x.cat.codes)
#data_com2= data_com2.drop(columns="MatlMaker")    
data_com2.head()

,Model,LayerNum,PP,MatlMaker,RC,MatlG,Cu1,CP1,Cu2,CP2,Input_T,Thickness,MatlMaker_cate
0,1,1,1,DS7402M,61.0,46.8,8.21,55,14.29,73,60,67.70,1
1,1,2,0,DS7402M,0.0,0.0,14.29,73,15.04,56,75,73.71,1
2,1,3,1,DS7402M,47.0,203.4,15.04,56,14.58,46,200,187.25,1
3,1,4,0,DS7402M,0.0,0.0,14.58,46,14.29,75,75,77.47,1
4,1,5,1,DS7402M,53.0,81.4,14.29,75,15.04,48,90,80.48,1


In [33]:
### total 용으로 
model_list_2 = list(data_com2["Model"].value_counts().keys())


testset_dic_2 = {}

### test_list:  뽑힌 정보의 index 정보를 저장하기 위해서 이용
#blinde testset으로 뽑힌 데이터는 train 할때 제거해야 하기 때문에,

testset_list_2 = []
for mo in model_list_2:
    ### 각 model의 testset을 model_(n) 이름의 변수로 저장하기
    """
    사용자함수: extraction_dataset
    """
    globals () ["model_{}_total".format(mo)] = extraction_dataset(data_com2,mo)
    testset_list_2.append(globals () ["model_{}_total".format(mo)])
    testset_dic_2["model_{}_total".format(mo)] = globals () ["model_{}".format(mo)]

In [34]:
## 예시: model_1 의 testset 불러오기
testset_dic["model_1"]

,Model,LayerNum,PP,MatlMaker,RC,MatlG,Cu1,CP1,Cu2,CP2,Input_T,Thickness
0,1,1,1,DS7402M,61.0,46.8,8.21,55,14.29,73,60,67.70
1,1,2,0,DS7402M,0.0,0.0,14.29,73,15.04,56,75,73.71
2,1,3,1,DS7402M,47.0,203.4,15.04,56,14.58,46,200,187.25
3,1,4,0,DS7402M,0.0,0.0,14.58,46,14.29,75,75,77.47
4,1,5,1,DS7402M,53.0,81.4,14.29,75,15.04,48,90,80.48
5,1,6,0,DS7402M,0.0,0.0,15.04,48,14.29,48,180,173.75
6,1,7,1,DS7402M,53.0,81.4,14.29,48,14.29,75,90,81.44
7,1,8,0,DS7402M,0.0,0.0,14.29,75,15.04,50,75,73.71
8,1,9,1,DS7402M,47.0,203.4,15.04,50,15.80,60,200,193.23
9,1,10,0,DS7402M,0.0,0.0,15.80,60,15.04,73,75,75.97


In [35]:
### train 할 데이터에서 Blind에서 뽑힌 데이터 제거하기
# 위의 testset_list 이용
drop_index = []
for i in testset_list:
    drop_index.extend(list(i.index))
data_drop = data_com.drop(drop_index).reset_index(drop=True)

### 제거된 데이터 = data_drop: 학습에 이용

In [36]:
data_drop.head()

,Model,LayerNum,PP,MatlMaker,RC,MatlG,Cu1,CP1,Cu2,CP2,Input_T,Thickness
0,1,1,1,DS7402M,61.0,46.8,8.37,55,14.29,73,60,66.19
1,1,2,0,DS7402M,0.0,0.0,14.29,73,15.04,56,75,74.47
2,1,3,1,DS7402M,47.0,203.4,15.04,56,15.04,46,200,196.32
3,1,4,0,DS7402M,0.0,0.0,15.04,46,15.04,75,75,75.22
4,1,5,1,DS7402M,53.0,81.4,15.04,75,15.04,48,90,82.26


In [37]:
data_total = data_drop.copy()
data_total["Model"] = "0"
data_total["MatlMaker"] = "All"
data_total.head()

,Model,LayerNum,PP,MatlMaker,RC,MatlG,Cu1,CP1,Cu2,CP2,Input_T,Thickness
0,0,1,1,All,61.0,46.8,8.37,55,14.29,73,60,66.19
1,0,2,0,All,0.0,0.0,14.29,73,15.04,56,75,74.47
2,0,3,1,All,47.0,203.4,15.04,56,15.04,46,200,196.32
3,0,4,0,All,0.0,0.0,15.04,46,15.04,75,75,75.22
4,0,5,1,All,53.0,81.4,15.04,75,15.04,48,90,82.26


In [38]:
### 이번에는 pp층만 불러오기
data_pp = data_drop[data_drop["PP"]==1]
data_pp.head()

,Model,LayerNum,PP,MatlMaker,RC,MatlG,Cu1,CP1,Cu2,CP2,Input_T,Thickness
0,1,1,1,DS7402M,61.0,46.8,8.37,55,14.29,73,60,66.19
2,1,3,1,DS7402M,47.0,203.4,15.04,56,15.04,46,200,196.32
4,1,5,1,DS7402M,53.0,81.4,15.04,75,15.04,48,90,82.26
6,1,7,1,DS7402M,53.0,81.4,14.58,48,14.58,75,90,80.48
8,1,9,1,DS7402M,47.0,203.4,15.80,50,15.04,60,200,202.85


In [39]:
def linear_reg(i,j,X_train,y_train,X_test,y_test):
    """
    i: Maker이름
    j: Glass이름
    나머지= data
    """
    # 충돌을 막기위해서 module 초기화 하기
    tf.keras.backend.clear_session()
    """
LR_DS7402M_48_8(:원래 46.8이지만, python에서 "."은 다른 의미로 사용되고 있어서, "_"로 변경
    """
    
    ### LR_Maker_Glass 이름으로 된 model 만들기
    globals()['LR_{}_{}'.format(i,str(j).replace(".","_"))] = linear_model.LinearRegression()
    # 모델 학습하기
    globals()['LR_{}_{}'.format(i,str(j).replace(".","_"))].fit(X_train, y_train)
    # 학습한 모델로 값 예측하기
    y_pred = globals()['LR_{}_{}'.format(i,str(j).replace(".","_"))].predict(X_test)
    """
    사용자함수: rsq
    """
    # R_square 값 계산하기
    rsqr_ = rsq(y_pred,y_test)
    # MSE 계산하기
    mse_ = mean_squared_error(y_test, y_pred)
    
    # 기타 기초통계 값 산출하기
    min_ = round(min(abs(y_test - y_pred)),3)
    max_ = round(max(abs(y_test - y_pred)),3)
    mean_ = round((abs(y_test - y_pred)).mean(),3)
    std_ = round(statistics.stdev(y_test - y_pred),3)
    return rsqr_,mse_,y_test,y_pred,[min_,max_,mean_,std_]

In [40]:
### 사용자함수: 기본 통계값을 산출해주는 함수
def print_statis(y_pred,y_test):
    rsqr_ = rsq(y_pred,y_test)
    min_ = round(min(abs(y_test - y_pred)),3)
    max_ = round(max(abs(y_test - y_pred)),3)
    mean_ = round((abs(y_test - y_pred)).mean(),3)
    std_ = round(statistics.stdev(y_test - y_pred),3)
    print("R SQR :" , rsqr_)        
    print("Min :" , min_)
    print("Max :" , max_)
    print("Mean : " , mean_)
    print("Std : ", std_)

In [41]:
def ann_model(i,j,X_train,y_train,X_test,y_test):
    """
    i: Maker이름
    j: Glass이름
    나머지= data
    """
    # 충돌을 막기위해서 module 초기화 하기
    tf.keras.backend.clear_session()
    
    ### ANN_Maker_Glass 이름으로 알고리즘 만들기
    # Basic Structure
    globals()['ANN_{}_{}'.format(i,str(j).replace(".","_"))] = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=[len(X_train.keys())]),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
      ])
    # Optimizer
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    # Loss function
    globals()['ANN_{}_{}'.format(i,str(j).replace(".","_"))].compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
    # 학습하기 (epoch= 300번으로)
    history = globals()['ANN_{}_{}'.format(i,str(j).replace(".","_"))].fit(X_train, y_train, epochs=300,verbose=0)
    
    # 학습결과 dataframe으로 저장하기
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    
    ### 알고리즘 평가
    # loss, mae, mse 값 도출
    loss_, mae_, mse_ = globals()['ANN_{}_{}'.format(i,str(j).replace(".","_"))].evaluate(X_test, y_test, verbose=2)
    # 값 예측하기
    y_pred = globals()['ANN_{}_{}'.format(i,str(j).replace(".","_"))].predict(X_test).flatten()
    """
    사용자함수: rsq
    """
    rsqr_ = rsq(y_pred,y_test)
    # 기초 통계값
    min_ = round(min(abs(y_test - y_pred)),3)
    max_ = round(max(abs(y_test - y_pred)),3)
    mean_ = round((abs(y_test - y_pred)).mean(),3)
    std_ = round(statistics.stdev(y_test - y_pred),3)
    return rsqr_,mse_,y_test,y_pred,[min_,max_,mean_,std_]    

In [42]:
### 사용자 함수: Linear regression, ANN 같이 돌리는 함수
def anal(data_drop,Drop_col,Graph_show):
    """
    data_drop: 데이터
    Drop_col: 데이터에서 제거해야 하는  columns 목록
    Graph_show: (yes,no) yes면 학습된 결과 graph로 표현
    """
    # Data 에 있는 Maker 목록
    Maker = data_drop['MatlMaker'].unique()
    # Data 에 있는 Glass 목록
    Glass_all = data_drop['MatlG'].unique()
    all_lr_2 = {}
    all_ann_2 = {}

    for i in Maker:
        # data 에서 i Maker만 뽑아오기
        df = data_drop[data_drop['MatlMaker'] == i]
        # 뽑은 데이터에 있는 Glass 종류
        Glass = df['MatlG'].unique()
        for j in Glass:   
            # 해당 Maker의 j의 Glass 만 뽑아내기
            df2 = df[df['MatlG'] == j]
            # input 으로 받은 분석에 필요 없는 columns 제거하기
            df3 = df2.drop(columns=Drop_col)
            # x,y 나누기
            y = df3.loc[:,"Thickness"]
            x = df3.drop(columns="Thickness")
            # train, test 나누기
            X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)
            """
            사용자함수: linear_reg (LR 학습)
            """
            rsqr_LR,mse_LR,y_test_LR,y_pred_LR,statis_LR = linear_reg(i,j,X_train,y_train,X_test,y_test)
            """
            사용자함수: ann_model (ANN 학습)
            """
            rsqr_ANN,mse_ANN,y_test_ANN,y_pred_ANN,statis_ANN = ann_model(i,j,X_train,y_train,X_test,y_test)
            
            # 결과값 저장하기
            all_lr_2["LR_{}_{}".format(i,j)]= [rsqr_LR,mse_LR,statis_LR[0],statis_LR[1],statis_LR[2],statis_LR[3]]
            all_ann_2["ANN_{}_{}".format(i,j)]= [rsqr_ANN,mse_ANN,statis_ANN[0],statis_ANN[1],statis_ANN[2],statis_ANN[3]]
            
            if Graph_show=="yes":
                print("#"*50,"{}_{}".format(i,j),"#"*50)
                display(" "*40,"LR"," "*40)
                """
                사용자함수: make_graph
                """
                make_graph(y_test_LR,y_pred_LR)
                """
                사용자함수: print_statis
                """
                print_statis(y_test_LR,y_pred_LR)
                display(" "*40,"ANN"," "*40)
                """
                사용자함수: make_graph
                """
                make_graph(y_test_ANN,y_pred_ANN)
                """
                사용자함수: print_statis
                """
                print_statis(y_test_ANN,y_pred_ANN)
            
                print("-"*120)
            else:
                pass
    # 결과값 저장
    hist_lr_2 = pd.DataFrame(all_lr_2).T
    hist_lr_2 = hist_lr_2.rename(columns={0:"R square",1:"MSE",2:"Min",3:"Max",4:"Mean",5:"Std"})

    hist_ann_2 = pd.DataFrame(all_ann_2).T
    hist_ann_2 = hist_ann_2.rename(columns={0:"R square",1:"MSE",2:"Min",3:"Max",4:"Mean",5:"Std"})
    return hist_lr_2,hist_ann_2

In [43]:
drop_list = ['MatlMaker','MatlG',"Model","PP"]
hist_lr,hist_ann = anal(data_pp,drop_list,"no")

2/2 - 0s - loss: 12.3709 - mae: 2.6653 - mse: 12.3709
2/2 - 0s - loss: 26.2599 - mae: 3.7579 - mse: 26.2599
1/1 - 0s - loss: 7.3296 - mae: 2.4283 - mse: 7.3296
1/1 - 0s - loss: 6.5353 - mae: 1.9939 - mse: 6.5353
2/2 - 0s - loss: 55.1959 - mae: 6.7989 - mse: 55.1959
1/1 - 0s - loss: 24.0223 - mae: 4.3196 - mse: 24.0223
1/1 - 0s - loss: 12.0186 - mae: 2.9097 - mse: 12.0186
1/1 - 0s - loss: 6.7806 - mae: 2.1337 - mse: 6.7806
1/1 - 0s - loss: 7.5455 - mae: 2.4856 - mse: 7.5455


1/1 - 0s - loss: 3.8996 - mae: 1.7404 - mse: 3.8996
1/1 - 0s - loss: 44.8208 - mae: 6.6516 - mse: 44.8208
1/1 - 0s - loss: 4.6003 - mae: 2.0168 - mse: 4.6003
1/1 - 0s - loss: 8.1397 - mae: 2.4045 - mse: 8.1397
1/1 - 0s - loss: 2.7911 - mae: 1.4520 - mse: 2.7911
1/1 - 0s - loss: 3.2466 - mae: 1.4314 - mse: 3.2466


1/1 - 0s - loss: 3.5768 - mae: 1.7727 - mse: 3.5768
1/1 - 0s - loss: 3.3310 - mae: 1.7784 - mse: 3.3310
1/1 - 0s - loss: 1.7751 - mae: 1.3106 - mse: 1.7751
1/1 - 0s - loss: 5.6175 - mae: 1.8329 - mse: 5.6175
1/1 - 0s - loss: 109.0633 - mae: 9.5593 - mse: 109.0633


1/1 - 0s - loss: 18.8460 - mae: 4.0818 - mse: 18.8460
1/1 - 0s - loss: 38.5626 - mae: 5.9880 - mse: 38.5626


In [54]:
hist_ann

,R square,MSE,Min,Max,Mean,Std
ANN_DS7402M_46.8,0.622758,12.370939,0.043,10.296,2.665,3.406
ANN_DS7402M_203.4,0.857187,26.259924,0.009,13.570,3.758,5.187
ANN_DS7402M_81.4,0.904751,7.329594,0.236,3.928,2.428,1.280
ANN_DS7402M_165.0,0.853180,6.535273,0.117,5.751,1.994,2.632
ANN_DS7402M_31.0,-0.493646,55.195881,0.556,11.970,6.799,3.536
ANN_DS7402M_103.8,0.768050,24.022308,0.850,8.464,4.320,3.002
ANN_DS7402M_24.4,0.092830,12.018620,0.292,5.413,2.910,2.221
ANN_NPG-151_46.8,-0.174718,6.780555,0.379,4.621,2.134,2.174
ANN_EM-370Z_46.8,0.847225,7.545461,1.039,4.504,2.486,2.266
ANN_EM-370Z_103.8,-0.054530,3.899554,0.141,2.939,1.740,1.952


In [57]:
makers,glasses = [],[]
for index_ in hist_ann.index:
    maker_,glass_= index_.split("_")[1:]
    makers.append(maker_)
    glasses.append(glass_)


In [64]:
mean_va = []
for i in hist_ann["Mean"]:
    mean_va.append(i)
mean_va

[2.665,
 3.758,
 2.428,
 1.994,
 6.799,
 4.32,
 2.91,
 2.134,
 2.486,
 1.74,
 6.652,
 2.017,
 2.404,
 1.452,
 1.431,
 1.773,
 1.778,
 1.311,
 1.833,
 9.559,
 4.082,
 5.988]

In [68]:
customs = []
for i in range(len(makers)):
    maker_ = makers[i]
    glass_ = float(glasses[i])
    print(maker_,glass_)
    custom_ = ANN_customize(data_drop,maker_,glass_)
    customs.append(custom_)

DS7402M 46.8


  0%|          | 0/100 [00:00<?, ?it/s]

DS7402M 203.4


  0%|          | 0/100 [00:00<?, ?it/s]

DS7402M 81.4


...

  0%|          | 0/100 [00:00<?, ?it/s]

EM-526 46.8


  0%|          | 0/100 [00:00<?, ?it/s]

EM-526 103.8


  0%|          | 0/100 [00:00<?, ?it/s]

In [70]:
(pd.DataFrame([makers,glasses,mean_va,customs]).T).rename(columns={0:"Maker",1:"Glass",2:"Mean",3:"Customize"})

,Maker,Glass,Mean,Customize
0,DS7402M,46.8,2.665,0.465724
1,DS7402M,203.4,3.758,0.68991
2,DS7402M,81.4,2.428,0.561181
3,DS7402M,165.0,1.994,0.689664
4,DS7402M,31.0,6.799,0.379479
5,DS7402M,103.8,4.32,0.633192
6,DS7402M,24.4,2.91,0.361414
7,NPG-151,46.8,2.134,0.465795
8,EM-370Z,46.8,2.486,0.465745
9,EM-370Z,103.8,1.74,0.633518
